In [8]:
#Import package
import random
import math
import numpy as np
import xlsxwriter
import pandas as pd

from pulp import *

# Preliminary model

In [9]:
# Define the model
model = LpProblem(name="Treatments_allocation_Problem", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0, cat="Integer") for i in range(1, 25)} # x[i] >=0 and integer (i = 1,2,3...24)

#Add constraints 
#supply constrains 
model += (x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] <= 2000, "bed constrain")
model += (x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] <= 600, "normal ward")
model += (x[17]+ x[18] + x[19] + x[20] + x[21] + x[22] + x[23] + x[24] <= 60, "ICU")

# demand constrains
model += (x[1] + x[9] + x[7] <= 8000, "non-con/m/~50") 
model += (x[2] + x[10] + x[18] <= 300, "non-con/m/50~")
model += (x[3] + x[11] + x[19] <= 1000, "con/m/~50")
model += (x[4] + x[12] + x[20] <= 50, "con/m/50~")
model += (x[5] + x[13] + x[21] <= 700, "non-con/s/~50")
model += (x[6] + x[14] + x[22] <= 30, "non-con/s/50~")
model += (x[7] + x[15] + x[23] <= 500, "con/s/~50")
model += (x[8] + x[16] + x[24] <= 20, "con/s/50~")
   
#Set the objective
model += x[1] + 3* x[3] + 2* x[7]+ x[9] + 2*x[11] + 2*x[12] + x[13] + 2*x[14] + x[15] + 2*x[16] + 3* x[17] + 3*x[18] + 3*x[20]
#Solve the optimization problem
status = model.solve()

#Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for var in x.values():
    print(f"{var.name}: {var.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")
    
# Sensitive analysis
sr = [{'Constraint Names':cname, 'Slack Values':cinfo.slack, 'Shadow Price':cinfo.pi} for cname, cinfo in model.constraints.items()] # slack values: how many supply or demand left at the end. Shadow price:if we have one more supply or demand availabel and give it to the constrain, our model benefit will increase the value of shadow price(in our model, all is 0)
print(pd.DataFrame(sr))

status: 1, Optimal
objective: 5380.0
x1: 500.0
x2: 0.0
x3: 1000.0
x4: 0.0
x5: 0.0
x6: 0.0
x7: 500.0
x8: 0.0
x9: 500.0
x10: 0.0
x11: 0.0
x12: 50.0
x13: 0.0
x14: 30.0
x15: 0.0
x16: 20.0
x17: 0.0
x18: 60.0
x19: 0.0
x20: 0.0
x21: 0.0
x22: 0.0
x23: 0.0
x24: 0.0
bed_constrain: 0.0
normal_ward: 0.0
ICU: 0.0
non_con/m/~50: -6500.0
non_con/m/50~: -240.0
con/m/~50: 0.0
con/m/50~: 0.0
non_con/s/~50: -700.0
non_con/s/50~: 0.0
con/s/~50: 0.0
con/s/50~: 0.0
   Constraint Names  Slack Values  Shadow Price
0     bed_constrain          -0.0          -0.0
1       normal_ward          -0.0          -0.0
2               ICU          -0.0          -0.0
3     non_con/m/~50        6500.0          -0.0
4     non_con/m/50~         240.0          -0.0
5         con/m/~50          -0.0          -0.0
6         con/m/50~          -0.0          -0.0
7     non_con/s/~50         700.0          -0.0
8     non_con/s/50~          -0.0          -0.0
9         con/s/~50          -0.0          -0.0
10        con/s/50~     

# Load  file

In [10]:
# load the dataset
df = pd.read_excel('supply_demand.xlsx') 
# set the index equal to the 's/d'
df.index = df['s/d']
df = df.drop('s/d',axis =1)
df = df.fillna(1)
# get the sum of all demand(use it later)
col = list(df)
col.remove('Supply')

sum_demand = df[col].sum(axis=1)[5]

# Final Linear Programming: Optimization With Python

In [11]:
# Define the model
model = LpProblem(name="Treatments_allocation_Problem", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0, cat="Integer") for i in range(1, 91)} # x[i] >=0 and integer (i = 1,2,3...90)

#Add constraints 
#supply constrains 
model += (x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] + x[17] + x[18] <= df['Supply'][0], "bed constrain")
model += (x[19] + x[20] + x[21] + x[22] + x[23] + x[24] + x[25] + x[26] + x[27] + x[28] + x[29] + x[30] + x[31] + x[32] + x[33] + x[34] + x[35] + x[36] <= df['Supply'][1], "normal ward constrain")
model += (x[37] + x[38] + x[39] + x[40] + x[41] + x[42] + x[43] + x[44] + x[45] + x[46] + x[47] + x[48] + x[49] + x[50] + x[51] + x[52] + x[53] + x[54] <= df['Supply'][2], "intensive therapy ward constrain")
model += (x[55] + x[56] + x[57] + x[58] + x[59] + x[60] + x[61] + x[62] + x[63] + x[64] + x[65] + x[66] + x[67] + x[68] + x[69] + x[70] + x[71] + x[72] <= df['Supply'][3], "ICU without ventilator constrain")
model += (x[73] + x[74] + x[75] + x[76] + x[77] + x[78] + x[79] + x[80] + x[81] + x[82] + x[83] + x[84] + x[85] + x[86] + x[87] + x[88] + x[89] + x[90] <= df['Supply'][4], "ICU with ventilator constrain")

# demand constrains
model += (x[1] + x[19] + x[37] + x[55] + x[73] <= df.loc['Demand',][17], "non-con/m/~65") 
model += (x[2] + x[20] + x[38] + x[56] + x[74] <= df.loc['Demand',][16], "non-con/m/65~85")
model += (x[3] + x[21] + x[39] + x[57] + x[75] <= df.loc['Demand',][15], "non-con/m/85~")
model += (x[4] + x[22] + x[40] + x[58] + x[76] <= df.loc['Demand',][14], "con/m/~65")
model += (x[5] + x[23] + x[41] + x[59] + x[77] <= df.loc['Demand',][13], "con/m/65~85")
model += (x[6] + x[24] + x[42] + x[60] + x[78] <= df.loc['Demand',][12], "con/m/85~")
model += (x[7] + x[25] + x[43] + x[61] + x[79] <= df.loc['Demand',][11], "lung/m/~65")
model += (x[8] + x[26] + x[44] + x[62] + x[80] <= df.loc['Demand',][10], "lung/m/65~85")
model += (x[9] + x[27] + x[45] + x[63] + x[81] <= df.loc['Demand',][9], "lung/m/85~")
model += (x[10] + x[28] + x[46] + x[64] + x[82] <= df.loc['Demand',][8], "non-con/s/~65")
model += (x[11] + x[29] + x[47] + x[65] + x[83] <= df.loc['Demand',][7], "non-con/s/65~85")
model += (x[12] + x[30] + x[48] + x[66] + x[84] <= df.loc['Demand',][6], "non-con/s/85~")
model += (x[13] + x[31] + x[49] + x[67] + x[85] <= df.loc['Demand',][5], "con/s/~65")
model += (x[14] + x[32] + x[50] + x[68] + x[86] <= df.loc['Demand',][4], "con/s/65~85")
model += (x[15] + x[33] + x[51] + x[69] + x[87] <= df.loc['Demand',][3], "con/s/85~")
model += (x[16] + x[34] + x[52] + x[70] + x[88] <= df.loc['Demand',][2], "lung/s/~65")
model += (x[17] + x[35] + x[53] + x[71] + x[89] <= df.loc['Demand',][1], "lung/s/65~85")
model += (x[18] + x[36] + x[54] + x[72] + x[90] <= df.loc['Demand',][0], "lung/s/85~")
   
#Set the objective
model += 3*x[11] + 2*x[12] + 3*x[13] + 2*x[14] + x[15] + 2*x[16] + x[17] + 3*x[24] + 3*x[26] + 3*x[27] + 3*x[28] + 2*x[29] + 2*x[31] + x[32] + x[34] + x[37] + x[38] + 3*x[39] + x[40] + 3*x[41] + 2*x[42] + 3*x[43] + 2*x[44] + x[45] + 2*x[46] + x[49] + 2*x[55] + 3*x[56] + 2*x[57] + 2*x[58] + 2*x[59] + 2*x[61] + x[62] + 3*x[73] + 2*x[74] + x[75] + 3*x[76] + x[77] + x[79]
#Solve the optimization problem
status = model.solve()

#Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
home = 0
for var in x.values():
    home += var.value()
    print(f"{var.name}: {var.value()}")
print("Stay at home:{}".format(sum_demand - home))
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")



status: 1, Optimal
objective: 5406.0
x1: 0.0
x2: 0.0
x3: 0.0
x4: 0.0
x5: 0.0
x6: 0.0
x7: 0.0
x8: 0.0
x9: 0.0
x10: 0.0
x11: 29.0
x12: 453.0
x13: 11.0
x14: 69.0
x15: 1104.0
x16: 49.0
x17: 315.0
x18: 0.0
x19: 0.0
x20: 0.0
x21: 0.0
x22: 0.0
x23: 0.0
x24: 276.0
x25: 0.0
x26: 245.0
x27: 95.0
x28: 0.0
x29: 0.0
x30: 0.0
x31: 0.0
x32: 0.0
x33: 0.0
x34: 0.0
x35: 0.0
x36: 0.0
x37: 0.0
x38: 0.0
x39: 113.0
x40: 0.0
x41: 0.0
x42: 0.0
x43: 17.0
x44: 70.0
x45: 0.0
x46: 5.0
x47: 0.0
x48: 0.0
x49: 0.0
x50: 0.0
x51: 0.0
x52: 0.0
x53: 0.0
x54: 0.0
x55: 0.0
x56: 29.0
x57: 0.0
x58: 0.0
x59: 69.0
x60: 0.0
x61: 32.0
x62: 0.0
x63: 0.0
x64: 0.0
x65: 0.0
x66: 0.0
x67: 0.0
x68: 0.0
x69: 0.0
x70: 0.0
x71: 0.0
x72: 0.0
x73: 5.0
x74: 0.0
x75: 0.0
x76: 11.0
x77: 0.0
x78: 0.0
x79: 0.0
x80: 0.0
x81: 0.0
x82: 0.0
x83: 0.0
x84: 0.0
x85: 0.0
x86: 0.0
x87: 0.0
x88: 0.0
x89: 0.0
x90: 0.0
Stay at home:8999.0
bed_constrain: -435.0
normal_ward_constrain: 0.0
intensive_therapy_ward_constrain: 0.0
ICU_without_ventilator_constrai

# Sensitive analysis: resourse(supply or demand) available change.

In [13]:
# Sensitive analysis
sr = [{'Constraint Names':cname, 'Slack Values':cinfo.slack, 'Shadow Price':cinfo.pi} for cname, cinfo in model.constraints.items()] # slack values: how many supply or demand left at the end. Shadow price:if we have one more supply or demand availabe and give it to the constrain, our model benefit will increase the value of shadow price(in our model, all is 0)
print(pd.DataFrame(sr))

                    Constraint Names  Slack Values  Shadow Price
0                      bed_constrain         435.0          -0.0
1              normal_ward_constrain          -0.0          -0.0
2   intensive_therapy_ward_constrain          -0.0          -0.0
3   ICU_without_ventilator_constrain          -0.0          -0.0
4      ICU_with_ventilator_constrain          51.0          -0.0
5                      non_con/m/~65          -0.0          -0.0
6                    non_con/m/65~85          -0.0          -0.0
7                      non_con/m/85~          -0.0          -0.0
8                          con/m/~65          -0.0          -0.0
9                        con/m/65~85          -0.0          -0.0
10                         con/m/85~          -0.0          -0.0
11                        lung/m/~65          -0.0          -0.0
12                      lung/m/65~85          -0.0          -0.0
13                        lung/m/85~        1159.0          -0.0
14                     no

# Sensitive analysis: objective function changed
## soultion : optimal solution change.

In [23]:
# Define the model
model = LpProblem(name="Treatments_allocation_Problem", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0, cat="Integer") for i in range(1, 91)} # x[i] >=0 and integer (i = 1,2,3...90)

#Add constraints 
#supply constrains 
model += (x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] + x[17] + x[18] <= df['Supply'][0], "bed constrain")
model += (x[19] + x[20] + x[21] + x[22] + x[23] + x[24] + x[25] + x[26] + x[27] + x[28] + x[29] + x[30] + x[31] + x[32] + x[33] + x[34] + x[35] + x[36] <= df['Supply'][1], "normal ward constrain")
model += (x[37] + x[38] + x[39] + x[40] + x[41] + x[42] + x[43] + x[44] + x[45] + x[46] + x[47] + x[48] + x[49] + x[50] + x[51] + x[52] + x[53] + x[54] <= df['Supply'][2], "intensive therapy ward constrain")
model += (x[55] + x[56] + x[57] + x[58] + x[59] + x[60] + x[61] + x[62] + x[63] + x[64] + x[65] + x[66] + x[67] + x[68] + x[69] + x[70] + x[71] + x[72] <= df['Supply'][3], "ICU without ventilator constrain")
model += (x[73] + x[74] + x[75] + x[76] + x[77] + x[78] + x[79] + x[80] + x[81] + x[82] + x[83] + x[84] + x[85] + x[86] + x[87] + x[88] + x[89] + x[90] <= df['Supply'][4], "ICU with ventilator constrain")

# demand constrains
model += (x[1] + x[19] + x[37] + x[55] + x[73] <= df.loc['Demand',][17], "non-con/m/~65") 
model += (x[2] + x[20] + x[38] + x[56] + x[74] <= df.loc['Demand',][16], "non-con/m/65~85")
model += (x[3] + x[21] + x[39] + x[57] + x[75] <= df.loc['Demand',][15], "non-con/m/85~")
model += (x[4] + x[22] + x[40] + x[58] + x[76] <= df.loc['Demand',][14], "con/m/~65")
model += (x[5] + x[23] + x[41] + x[59] + x[77] <= df.loc['Demand',][13], "con/m/65~85")
model += (x[6] + x[24] + x[42] + x[60] + x[78] <= df.loc['Demand',][12], "con/m/85~")
model += (x[7] + x[25] + x[43] + x[61] + x[79] <= df.loc['Demand',][11], "lung/m/~65")
model += (x[8] + x[26] + x[44] + x[62] + x[80] <= df.loc['Demand',][10], "lung/m/65~85")
model += (x[9] + x[27] + x[45] + x[63] + x[81] <= df.loc['Demand',][9], "lung/m/85~")
model += (x[10] + x[28] + x[46] + x[64] + x[82] <= df.loc['Demand',][8], "non-con/s/~65")
model += (x[11] + x[29] + x[47] + x[65] + x[83] <= df.loc['Demand',][7], "non-con/s/65~85")
model += (x[12] + x[30] + x[48] + x[66] + x[84] <= df.loc['Demand',][6], "non-con/s/85~")
model += (x[13] + x[31] + x[49] + x[67] + x[85] <= df.loc['Demand',][5], "con/s/~65")
model += (x[14] + x[32] + x[50] + x[68] + x[86] <= df.loc['Demand',][4], "con/s/65~85")
model += (x[15] + x[33] + x[51] + x[69] + x[87] <= df.loc['Demand',][3], "con/s/85~")
model += (x[16] + x[34] + x[52] + x[70] + x[88] <= df.loc['Demand',][2], "lung/s/~65")
model += (x[17] + x[35] + x[53] + x[71] + x[89] <= df.loc['Demand',][1], "lung/s/65~85")
model += (x[18] + x[36] + x[54] + x[72] + x[90] <= df.loc['Demand',][0], "lung/s/85~")
   
#Set the objective
#model += 0.3*x[11] + 2*x[12] + 3*x[13] + 2*x[14] + x[15] + 2*x[16] + x[17] + 3*x[24] + 3*x[26] + 3*x[27] + 3*x[28] + 2*x[29] + 2*x[31] + x[32] + x[34] + x[37] + x[38] + 3*x[39] + x[40] + 3*x[41] + 2*x[42] + 3*x[43] + 2*x[44] + x[45] + 2*x[46] + x[49] + 2*x[55] + 3*x[56] + 2*x[57] + 2*x[58] + 2*x[59] + 2*x[61] + x[62] + 3*x[73] + 2*x[74] + x[75] + 3*x[76] + x[77] + x[79]
#model += 300*x[11] + 2*x[12] + 3*x[13] + 2*x[14] + x[15] + 2*x[16] + x[17] + 3*x[24] + 3*x[26] + 3*x[27] + 3*x[28] + 2*x[29] + 2*x[31] + x[32] + x[34] + x[37] + x[38] + 3*x[39] + x[40] + 3*x[41] + 2*x[42] + 3*x[43] + 2*x[44] + x[45] + 2*x[46] + x[49] + 2*x[55] + 3*x[56] + 2*x[57] + 2*x[58] + 2*x[59] + 2*x[61] + x[62] + 3*x[73] + 2*x[74] + x[75] + 3*x[76] + x[77] + x[79]
#model += 3*x[11] + 200*x[12] + 3*x[13] + 2*x[14] + x[15] + 2*x[16] + x[17] + 3*x[24] + 3*x[26] + 3*x[27] + 3*x[28] + 2*x[29] + 2*x[31] + x[32] + x[34] + x[37] + x[38] + 3*x[39] + x[40] + 3*x[41] + 2*x[42] + 3*x[43] + 2*x[44] + x[45] + 2*x[46] + x[49] + 2*x[55] + 3*x[56] + 2*x[57] + 2*x[58] + 2*x[59] + 2*x[61] + x[62] + 3*x[73] + 2*x[74] + x[75] + 3*x[76] + x[77] + x[79]
model += 3*x[11] + 2*x[12] + 300*x[13] + 2*x[14] + x[15] + 2*x[16] + x[17] + 3*x[24] + 3*x[26] + 3*x[27] + 3*x[28] + 2*x[29] + 2*x[31] + x[32] + x[34] + x[37] + x[38] + 3*x[39] + x[40] + 3*x[41] + 2*x[42] + 3*x[43] + 2*x[44] + x[45] + 2*x[46] + x[49] + 2*x[55] + 3*x[56] + 2*x[57] + 2*x[58] + 2*x[59] + 2*x[61] + x[62] + 3*x[73] + 2*x[74] + x[75] + 3*x[76] + x[77] + x[79]
#Solve the optimization problem
status = model.solve()

#Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
home = 0
for var in x.values():
    home += var.value()
    print(f"{var.name}: {var.value()}")
print("Stay at home:{}".format(sum_demand - home))
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

# Sensitive analysis
sr = [{'Constraint Names':cname, 'Slack Values':cinfo.slack, 'Shadow Price':cinfo.pi} for cname, cinfo in model.constraints.items()] # slack values: how many supply or demand left at the end. Shadow price:if we have one more supply or demand availabel and give it to the constrain, our model benefit will increase the value of shadow price(in our model, all is 0)
print(pd.DataFrame(sr))

status: 1, Optimal
objective: 8673.0
x1: 0.0
x2: 0.0
x3: 0.0
x4: 0.0
x5: 0.0
x6: 0.0
x7: 0.0
x8: 0.0
x9: 0.0
x10: 0.0
x11: 29.0
x12: 453.0
x13: 11.0
x14: 69.0
x15: 1104.0
x16: 49.0
x17: 315.0
x18: 0.0
x19: 0.0
x20: 0.0
x21: 0.0
x22: 0.0
x23: 0.0
x24: 276.0
x25: 0.0
x26: 245.0
x27: 95.0
x28: 0.0
x29: 0.0
x30: 0.0
x31: 0.0
x32: 0.0
x33: 0.0
x34: 0.0
x35: 0.0
x36: 0.0
x37: 0.0
x38: 0.0
x39: 113.0
x40: 0.0
x41: 0.0
x42: 0.0
x43: 17.0
x44: 70.0
x45: 0.0
x46: 5.0
x47: 0.0
x48: 0.0
x49: 0.0
x50: 0.0
x51: 0.0
x52: 0.0
x53: 0.0
x54: 0.0
x55: 0.0
x56: 29.0
x57: 0.0
x58: 0.0
x59: 69.0
x60: 0.0
x61: 32.0
x62: 0.0
x63: 0.0
x64: 0.0
x65: 0.0
x66: 0.0
x67: 0.0
x68: 0.0
x69: 0.0
x70: 0.0
x71: 0.0
x72: 0.0
x73: 5.0
x74: 0.0
x75: 0.0
x76: 11.0
x77: 0.0
x78: 0.0
x79: 0.0
x80: 0.0
x81: 0.0
x82: 0.0
x83: 0.0
x84: 0.0
x85: 0.0
x86: 0.0
x87: 0.0
x88: 0.0
x89: 0.0
x90: 0.0
Stay at home:8999.0
bed_constrain: -435.0
normal_ward_constrain: 0.0
intensive_therapy_ward_constrain: 0.0
ICU_without_ventilator_constrai

# Sensitive analysis: add a new constrain
## constrain: The number of patients with a sever condition and larger than 85 years old are less than 40.
## solution: Optimal solution changed. x12 changed from 453 to 40.

In [15]:
# Define the model
model = LpProblem(name="Treatments_allocation_Problem", sense=LpMaximize)

# Define the decision variables
x = {i: LpVariable(name=f"x{i}", lowBound=0, cat="Integer") for i in range(1, 91)} # x[i] >=0 and integer (i = 1,2,3...90)

#Add constraints 
#supply constrains 
model += (x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] + x[17] + x[18] <= df['Supply'][0], "bed constrain")
model += (x[19] + x[20] + x[21] + x[22] + x[23] + x[24] + x[25] + x[26] + x[27] + x[28] + x[29] + x[30] + x[31] + x[32] + x[33] + x[34] + x[35] + x[36] <= df['Supply'][1], "normal ward constrain")
model += (x[37] + x[38] + x[39] + x[40] + x[41] + x[42] + x[43] + x[44] + x[45] + x[46] + x[47] + x[48] + x[49] + x[50] + x[51] + x[52] + x[53] + x[54] <= df['Supply'][2], "intensive therapy ward constrain")
model += (x[55] + x[56] + x[57] + x[58] + x[59] + x[60] + x[61] + x[62] + x[63] + x[64] + x[65] + x[66] + x[67] + x[68] + x[69] + x[70] + x[71] + x[72] <= df['Supply'][3], "ICU without ventilator constrain")
model += (x[73] + x[74] + x[75] + x[76] + x[77] + x[78] + x[79] + x[80] + x[81] + x[82] + x[83] + x[84] + x[85] + x[86] + x[87] + x[88] + x[89] + x[90] <= df['Supply'][4], "ICU with ventilator constrain")
model += (x[12]+x[30]+x[48]+x[66]+x[84]+x[15]+x[33]+x[51]+x[69] +x[87]+x[18]+x[36]+x[54]+x[72]+x[90] <= 40) # add a constrain
# demand constrains
model += (x[1] + x[19] + x[37] + x[55] + x[73] <= df.loc['Demand',][17], "non-con/m/~65") 
model += (x[2] + x[20] + x[38] + x[56] + x[74] <= df.loc['Demand',][16], "non-con/m/65~85")
model += (x[3] + x[21] + x[39] + x[57] + x[75] <= df.loc['Demand',][15], "non-con/m/85~")
model += (x[4] + x[22] + x[40] + x[58] + x[76] <= df.loc['Demand',][14], "con/m/~65")
model += (x[5] + x[23] + x[41] + x[59] + x[77] <= df.loc['Demand',][13], "con/m/65~85")
model += (x[6] + x[24] + x[42] + x[60] + x[78] <= df.loc['Demand',][12], "con/m/85~")
model += (x[7] + x[25] + x[43] + x[61] + x[79] <= df.loc['Demand',][11], "lung/m/~65")
model += (x[8] + x[26] + x[44] + x[62] + x[80] <= df.loc['Demand',][10], "lung/m/65~85")
model += (x[9] + x[27] + x[45] + x[63] + x[81] <= df.loc['Demand',][9], "lung/m/85~")
model += (x[10] + x[28] + x[46] + x[64] + x[82] <= df.loc['Demand',][8], "non-con/s/~65")
model += (x[11] + x[29] + x[47] + x[65] + x[83] <= df.loc['Demand',][7], "non-con/s/65~85")
model += (x[12] + x[30] + x[48] + x[66] + x[84] <= df.loc['Demand',][6], "non-con/s/85~")
model += (x[13] + x[31] + x[49] + x[67] + x[85] <= df.loc['Demand',][5], "con/s/~65")
model += (x[14] + x[32] + x[50] + x[68] + x[86] <= df.loc['Demand',][4], "con/s/65~85")
model += (x[15] + x[33] + x[51] + x[69] + x[87] <= df.loc['Demand',][3], "con/s/85~")
model += (x[16] + x[34] + x[52] + x[70] + x[88] <= df.loc['Demand',][2], "lung/s/~65")
model += (x[17] + x[35] + x[53] + x[71] + x[89] <= df.loc['Demand',][1], "lung/s/65~85")
model += (x[18] + x[36] + x[54] + x[72] + x[90] <= df.loc['Demand',][0], "lung/s/85~")
   
#Set the objective
model += 3*x[11] + 2*x[12] + 3*x[13] + 2*x[14] + x[15] + 2*x[16] + x[17] + 3*x[24] + 3*x[26] + 3*x[27] + 3*x[28] + 2*x[29] + 2*x[31] + x[32] + x[34] + x[37] + x[38] + 3*x[39] + x[40] + 3*x[41] + 2*x[42] + 3*x[43] + 2*x[44] + x[45] + 2*x[46] + x[49] + 2*x[55] + 3*x[56] + 2*x[57] + 2*x[58] + 2*x[59] + 2*x[61] + x[62] + 3*x[73] + 2*x[74] + x[75] + 3*x[76] + x[77] + x[79]
#Solve the optimization problem
status = model.solve()

#Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
home = 0
for var in x.values():
    home += var.value()
    print(f"{var.name}: {var.value()}")
print("Stay at home:{}".format(sum_demand - home))
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

# Sensitive analysis
sr = [{'Constraint Names':cname, 'Slack Values':cinfo.slack, 'Shadow Price':cinfo.pi} for cname, cinfo in model.constraints.items()] # slack values: how many supply or demand left at the end. Shadow price:if we have one more supply or demand availabel and give it to the constrain, our model benefit will increase the value of shadow price(in our model, all is 0)
print(pd.DataFrame(sr))

status: 1, Optimal
objective: 3476.0
x1: 0.0
x2: 0.0
x3: 0.0
x4: 0.0
x5: 0.0
x6: 0.0
x7: 0.0
x8: 0.0
x9: 0.0
x10: 0.0
x11: 29.0
x12: 40.0
x13: 11.0
x14: 69.0
x15: 0.0
x16: 49.0
x17: 315.0
x18: 0.0
x19: 0.0
x20: 0.0
x21: 0.0
x22: 0.0
x23: 0.0
x24: 276.0
x25: 0.0
x26: 245.0
x27: 95.0
x28: 0.0
x29: 0.0
x30: 0.0
x31: 0.0
x32: 0.0
x33: 0.0
x34: 0.0
x35: 0.0
x36: 0.0
x37: 0.0
x38: 0.0
x39: 113.0
x40: 0.0
x41: 0.0
x42: 0.0
x43: 17.0
x44: 70.0
x45: 0.0
x46: 5.0
x47: 0.0
x48: 0.0
x49: 0.0
x50: 0.0
x51: 0.0
x52: 0.0
x53: 0.0
x54: 0.0
x55: 0.0
x56: 29.0
x57: 0.0
x58: 0.0
x59: 69.0
x60: 0.0
x61: 32.0
x62: 0.0
x63: 0.0
x64: 0.0
x65: 0.0
x66: 0.0
x67: 0.0
x68: 0.0
x69: 0.0
x70: 0.0
x71: 0.0
x72: 0.0
x73: 5.0
x74: 0.0
x75: 0.0
x76: 11.0
x77: 0.0
x78: 0.0
x79: 0.0
x80: 0.0
x81: 0.0
x82: 0.0
x83: 0.0
x84: 0.0
x85: 0.0
x86: 0.0
x87: 0.0
x88: 0.0
x89: 0.0
x90: 0.0
Stay at home:10516.0
bed_constrain: -1952.0
normal_ward_constrain: 0.0
intensive_therapy_ward_constrain: 0.0
ICU_without_ventilator_constrain: